# CP2K Multistage and DDEC App

**Authors: Aliaksandr Yakutovich (LSMO/THEOS, EPFL)**

This automatic workflow allows to optimize geometry of a MOF/COF material
It is powered by:
- [CP2K](https://www.cp2k.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-CP2K](https://github.com/aiidateam/aiida-cp2k) plugin
- Custom-made workflows for AiiDA to manage the selection of parameters, the error handling, ...
- [AppMode for Jupyter](http://github.com/oschuett/jupyter_appmode) to create a simple UI

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import io
import ipywidgets as ipw
from IPython.display import clear_output
from traitlets import dlink
import urllib.parse as urlparse

# AiiDA imports.
%aiida
from aiida.orm import SinglefileData

# Base widgets.
from aiidalab_widgets_base import StructureBrowserWidget, StructureExamplesWidget, StructureManagerWidget
from aiidalab_widgets_base import BasicStructureEditor, StructureUploadWidget, OptimadeQueryWidget
from aiidalab_widgets_base import SubmitButtonWidget, CodeDropdown, ExportButtonWidget
from aiidalab_widgets_base import ProcessFollowerWidget, ProgressBarWidget, viewer

# Local imports.
from utils.mof_cleaner.editor import SolventOverlapCleaner
from utils.mof_checker import MofCheckerWidget, CheckMofStructure

# Workchains.
Cp2kMultistageDdecWorkChain = WorkflowFactory('lsmo.cp2k_multistage_ddec')
Cp2kMultistageWorkChain = WorkflowFactory('lsmo.cp2k_multistage')

In [ ]:
optimade_widget = OptimadeQueryWidget()
optimade_widget.title = "OPTIMADE"

structure_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        StructureBrowserWidget(title="From AiiDA database"),
        optimade_widget,
        StructureExamplesWidget(
            title="From Examples",
            examples=[
                ('Aluminium', 'data/Al.cif')
            ]),
    ],
    editors = [
        BasicStructureEditor(title="Basic Editor"),
        SolventOverlapCleaner(title="Framework Clearner"),
        MofCheckerWidget(title="MOF Checker"),  
    ],
    storable=True,
    node_class='StructureData'
)

cp2k_code = CodeDropdown(input_plugin='cp2k', text="CP2K code:")
ddec_code = CodeDropdown(input_plugin='ddec', text="DDEC code:")
number_of_nodes = ipw.IntText(value=1,
                              step=1,description = "that will be run on",
                              disabled=False,
                              layout=ipw.Layout(width="180px"),
                              style={"description_width":"120px"},)
cpus_per_node = ipw.IntText(
    value=1,
    step=1,
    description = "",
    disabled=False,
    layout=ipw.Layout(width="50px"),
    style={"description_width":"0px"},
)

protocol = ipw.ToggleButtons(
    options = [
        ('Standard', 'standard'),
        ('Test', 'test'),
        ('Single point', 'singlepoint'), 
        ('Robust convergence', 'robust_conv')
    ],
    description='Protocol:',
    style = {'description_width': 'initial'},
)

user_protocol = ipw.FileUpload(description='Your protocol', accept='.yaml', button_style='info')
user_defined = ipw.Checkbox(description="User-defined", value=False, layout={'margin': '0px 0px 0px -80px'})
dlink((user_defined, 'value'), (user_protocol, 'disabled'), transform=lambda v: not v)
dlink((user_defined, 'value'), (protocol, 'disabled'))


compute_charges = ipw.Checkbox(value=True, description='Compute point charges')

output = ipw.HTML('')

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
parsed_url = urlparse.parse_qs(url.query)
if 'structure_uuid' in parsed_url:
    structure_widget.input_structure = load_node(parsed_url['structure_uuid'][0])

In [ ]:
def setup_calc_multistage(builder=None):
    output.value = ''

    builder = builder or Cp2kMultistageWorkChain.get_builder()

    # Resources and walltime.
    builder.cp2k_base.cp2k.metadata.options =  {
        'max_wallclock_seconds': 3600 * 10,
        'resources':{
            'num_machines': number_of_nodes.value,
            'num_mpiprocs_per_machine': cpus_per_node.value,
        }
    }
    # Structure.
    if structure_widget.structure_node is None:
        output.value = "Please select a structure."
        return None
    builder.structure = structure_widget.structure_node

    # CP2K code.
    if cp2k_code.selected_code is None:
        output.value = "Please select CP2K code."
        return None
    builder.cp2k_base.cp2k.code = cp2k_code.selected_code

    # Protocol.
    if user_defined.value:
        for fname, item in user_protocol.value.items():
            fobj = io.BytesIO(item['content'])
            builder.protocol_yaml = SinglefileData(fobj)
    else:
        builder.protocol_tag = Str(protocol.value)

    return builder

def setup_calc_multistage_charges():
    builder = Cp2kMultistageDdecWorkChain.get_builder()
    
    # Setting the optimization inputs.
    if setup_calc_multistage(builder) is None:
        return None

    # Setting the charge computation inputs.
    builder.ddec.metadata.options =  {
        'max_wallclock_seconds': 3600 * 2,
        'withmpi': False,
        'resources':{
            'num_machines': 1,
        }
    }

    if ddec_code.selected_code is None:
        output.value = "Please select DDEC code."
        return None
    builder.ddec.code = ddec_code.selected_code

    # Parameters.
    builder.ddec.parameters = Dict(
        dict={
            'net charge': 0.0,
            'charge type': 'DDEC6',
            'periodicity along A, B, and C vectors': [True, True, True],
            'compute BOs': False,
            'atomic densities directory complete path': "/work/lsmo/aiida-lsmo-codes/data/chargemol/atomic_densities/",
            'input filename': 'valence_density',
        })

    return builder

In [ ]:
submit_widget_multistage_charges = SubmitButtonWidget(Cp2kMultistageDdecWorkChain, setup_calc_multistage_charges, description="Run GeoOpt + Point Charges")
submit_widget_multistage = SubmitButtonWidget(Cp2kMultistageWorkChain, setup_calc_multistage, description="Run GeoOpt")
submit_widget_multistage.btn_submit.layout = {"width": "240px"}
submit_widget_multistage_charges.btn_submit.layout = {"width": "240px"}
submit_buttons = ipw.Output()
def update_btns(_=None):
    with submit_buttons:
        clear_output()
        if compute_charges.value:
            display(submit_widget_multistage_charges)
            ddec_code.dropdown.disabled = False
        else:
            display(submit_widget_multistage)
            ddec_code.dropdown.disabled = True

update_btns()
compute_charges.observe(update_btns, 'value')

In [ ]:
check_structure_widget = CheckMofStructure()
_ = dlink((structure_widget, 'structure'), (check_structure_widget, 'structure'))

In [ ]:
display(structure_widget,
        check_structure_widget,
        ipw.HBox([cp2k_code, number_of_nodes, ipw.HTML("node(s)"), cpus_per_node, ipw.HTML("CPU each")]),
        ddec_code,
        ipw.HBox([protocol, user_protocol, user_defined]),
        compute_charges, submit_buttons, output)